# Orchestrator Task and Dependency Visualization

This notebook visualizes the tasks created and their dependencies for all combinations of STARTING_LEVEL and ENDING_LEVEL.
It also shows which folders are created and when basin folders are cleaned before Level 4.

In [1]:
import sys
import pandas as pd
from pathlib import Path
from collections import defaultdict
import json

# Add source directory to path
sys.path.insert(0, '/ihme/homes/bcreiner/repos/idd-climate-models/src')

import idd_climate_models.constants as rfc
from idd_climate_models.io_compare_utils import compare_model_validation
from idd_climate_models.dictionary_utils import parse_results

print("Imports successful!")

Imports successful!


In [2]:
# Configuration (matching orchestrator)
DATA_SOURCE = "cmip6"
NUM_DRAWS = rfc.NUM_DRAWS
TEST_RUN = 2  # Just test with 2 tasks for fast execution
ADD_DEPENDENCIES = True
BASINS = ['EP', 'NA', 'NI', 'SI', 'AU', 'SP', 'WP']
VERBOSE = True
DELETE_EXISTING_FOLDERS = True
CLEAN_BASIN_FOLDERS_BEFORE_LEVEL4 = True

# Validation configuration
INPUT_DATA_TYPE = "data"
INPUT_IO_TYPE = "raw"
RAW_DATA_PATH = rfc.RAW_DATA_PATH
TC_RISK_INPUT_PATH = rfc.TC_RISK_INPUT_PATH
TC_RISK_OUTPUT_PATH = rfc.TC_RISK_OUTPUT_PATH
TIME_BINS_DF_PATH = rfc.TIME_BINS_DF_PATH

print(f"NUM_DRAWS = {NUM_DRAWS}")
print(f"TC_RISK_OUTPUT_PATH = {TC_RISK_OUTPUT_PATH}")

NUM_DRAWS = 250
TC_RISK_OUTPUT_PATH = /mnt/team/rapidresponse/pub/tropical-storms/tc_risk/output


In [3]:
# Load time bins
time_bins_df = pd.read_csv(TIME_BINS_DF_PATH)
time_bins_df = time_bins_df[time_bins_df['method'] == 'BayesPoisson']
time_bins_df['time_period'] = time_bins_df['start_year'].astype(str) + '-' + time_bins_df['end_year'].astype(str)
unique_time_bins = time_bins_df[['model', 'variant', 'scenario', 'time_period', 'start_year', 'end_year']].drop_duplicates()

print(f"Loaded {len(unique_time_bins)} unique time bins")

# Create lookup
time_bins_lookup = {}
for _, row in unique_time_bins.iterrows():
    key = (row['model'], row['variant'], row['scenario'])
    if key not in time_bins_lookup:
        time_bins_lookup[key] = []
    time_bins_lookup[key].append({
        'time_period': row['time_period'],
        'start_year': row['start_year'],
        'end_year': row['end_year']
    })

print(f"Found {len(time_bins_lookup)} unique model/variant/scenario combinations")

Loaded 130 unique time bins
Found 32 unique model/variant/scenario combinations


In [4]:
# Validate raw data (only for demo)
print("Validating raw data...")
validation_info = compare_model_validation(
    input_data_type=INPUT_DATA_TYPE,
    input_io_type=INPUT_IO_TYPE,
    output_data_type="tc_risk",
    output_io_type="input",
    data_source=DATA_SOURCE,
    verbose=False,
    rerun_all=True,
)

input_complete_model_variants = validation_info["input_complete_model_variants"]
full_path_list = parse_results(validation_info["input_validation_dict"], 'all')

# Build variable detail map
variable_detail_map = {}
for item in full_path_list:
    key = (item['model'], item['variant'], item['scenario'], item['variable'])
    variable_detail_map[key] = {
        'grid': item['grid'],
        'frequency': item['frequency'],
    }

print(f"Found {len(input_complete_model_variants)} complete model/variant combinations")

Validating raw data...

Validation complete for: data, raw, cmip6
Summary: 4/4 models complete. Parsed log (up to 'grid') written to /mnt/team/rapidresponse/pub/tropical-storms/data/raw/cmip6/validation_log.json
⚠️ Rerun all flag is set. All complete input model/variants will be reprocessed.
Found 8 complete model/variant combinations


In [5]:
# Helper functions for checking output status

def check_level3_output_finished(model, variant, scenario, time_period, data_source="cmip6"):
    """Check if Level 3 has both env_wnd and thermo files."""
    output_path = TC_RISK_OUTPUT_PATH / data_source / model / variant / scenario / time_period
    if not output_path.exists():
        return False
    env_wnd_files = list(output_path.glob("env_wnd_*.nc"))
    thermo_files = list(output_path.glob("thermo_*.nc"))
    return len(env_wnd_files) > 0 and len(thermo_files) > 0

def get_level4_basin_file_count(model, variant, scenario, time_period, basin, data_source="cmip6"):
    """Get file count in a basin folder."""
    output_path = TC_RISK_OUTPUT_PATH / data_source / model / variant / scenario / time_period / basin
    if not output_path.exists():
        return -1
    files = list(output_path.glob("*.nc"))
    return len(files)

def check_level4_output_complete(model, variant, scenario, time_period, basin, data_source="cmip6"):
    """Check if Level 4 has NUM_DRAWS + 1 files."""
    file_count = get_level4_basin_file_count(model, variant, scenario, time_period, basin, data_source)
    return file_count >= NUM_DRAWS + 1

print("Helper functions defined!")

Helper functions defined!


In [6]:
# Build task list
tasks_to_run = []

for model_variant_tuple in input_complete_model_variants:
    model_name, variant_name = model_variant_tuple
    for scenario in rfc.SCENARIOS:
        key = (model_name, variant_name, scenario)
        if key not in time_bins_lookup:
            continue
        time_periods = time_bins_lookup[key]
        for tp_info in time_periods:
            tasks_to_run.append({
                'model': model_name,
                'variant': variant_name,
                'scenario': scenario,
                'time_period': tp_info['time_period'],
            })

# Apply test run limit
if TEST_RUN > 0:
    tasks_to_run = tasks_to_run[:TEST_RUN]

print(f"Total tasks to simulate: {len(tasks_to_run)}")
print(f"\nFirst {min(3, len(tasks_to_run))} tasks:")
for task in tasks_to_run[:3]:
    print(f"  {task['model']}/{task['variant']}/{task['scenario']}/{task['time_period']}")

Total tasks to simulate: 2

First 2 tasks:
  CMCC-ESM2/r1i1p1f1/historical/1970-1986
  CMCC-ESM2/r1i1p1f1/historical/1987-2001


In [7]:
def simulate_orchestrator(starting_level, ending_level, test_tasks=None):
    """
    Simulate the orchestrator logic and return a detailed report of:
    - Tasks created at each level
    - Dependencies between tasks
    - Folder creation operations
    - Cleaning operations
    """
    
    if test_tasks is None:
        test_tasks = tasks_to_run
    
    report = {
        'starting_level': starting_level,
        'ending_level': ending_level,
        'tasks_by_level': {
            1: {'count': 0, 'tasks': []},
            2: {'count': 0, 'tasks': []},
            3: {'count': 0, 'tasks': []},
            4: {'count': 0, 'tasks': [], 'cleaned_basins': []},
        },
        'folder_operations': [],
        'dependencies': [],
        'level3_status': {'finished': 0, 'not_finished': 0},
        'level4_basin_status': {'complete': 0, 'incomplete': 0},
    }
    
    # Track tasks created for each level
    level1_tasks = []
    level2_tasks = []
    level3_tasks = []
    level3_tasks_dict = {}  # For tracking dependencies
    level4_basin_tasks = []
    
    # ========== LEVEL 1: Folder Creation ==========
    if starting_level <= 1 and ending_level >= 1:
        for task in test_tasks:
            folder_task_name = f"FOLDER_{task['model']}_{task['variant']}_{task['scenario']}_{task['time_period']}"
            level1_tasks.append({
                'name': folder_task_name,
                'model': task['model'],
                'variant': task['variant'],
                'scenario': task['scenario'],
                'time_period': task['time_period'],
            })
            
            # Record folder creation
            path = TC_RISK_INPUT_PATH / DATA_SOURCE / task['model'] / task['variant'] / task['scenario'] / task['time_period']
            report['folder_operations'].append(f"CREATE: {path}")
            path_output = TC_RISK_OUTPUT_PATH / DATA_SOURCE / task['model'] / task['variant'] / task['scenario'] / task['time_period']
            report['folder_operations'].append(f"CREATE: {path_output}")
        
        report['tasks_by_level'][1]['count'] = len(level1_tasks)
        report['tasks_by_level'][1]['tasks'] = level1_tasks
    
    # ========== LEVEL 2: Process Variables ==========
    if starting_level <= 2 and ending_level >= 2:
        for task in test_tasks:
            model_name = task['model']
            variant_name = task['variant']
            scenario_name = task['scenario']
            time_period_str = task['time_period']
            
            for variable in rfc.VARIABLES[DATA_SOURCE]:
                detail_key = (model_name, variant_name, scenario_name, variable)
                if detail_key not in variable_detail_map:
                    continue
                
                details = variable_detail_map[detail_key]
                process_task_name = f"PROCESS_{model_name}_{variant_name}_{scenario_name}_{time_period_str}_{variable}"
                level2_tasks.append({
                    'name': process_task_name,
                    'model': model_name,
                    'variant': variant_name,
                    'scenario': scenario_name,
                    'time_period': time_period_str,
                    'variable': variable,
                    'grid': details['grid'],
                    'frequency': details['frequency'],
                })
                
                # Dependency: Level 2 depends on Level 1
                if ADD_DEPENDENCIES and starting_level <= 1:
                    folder_task_name = f"FOLDER_{model_name}_{variant_name}_{scenario_name}_{time_period_str}"
                    report['dependencies'].append({
                        'child': process_task_name,
                        'parent': folder_task_name,
                        'level': 'L2 -> L1'
                    })
        
        report['tasks_by_level'][2]['count'] = len(level2_tasks)
        report['tasks_by_level'][2]['tasks'] = level2_tasks
    
    # ========== LEVEL 3: Check Output Status ==========
    if starting_level >= 3:
        level3_finished = []
        level3_not_finished = []
        
        for task in test_tasks:
            if check_level3_output_finished(task['model'], task['variant'], task['scenario'], task['time_period'], DATA_SOURCE):
                level3_finished.append(task)
            else:
                level3_not_finished.append(task)
        
        report['level3_status']['finished'] = len(level3_finished)
        report['level3_status']['not_finished'] = len(level3_not_finished)
        
        # Determine which tasks to create based on STARTING_LEVEL
        if starting_level == 3:
            level3_tasks_to_create = level3_not_finished
            level4_tasks_to_check = test_tasks  # Check all for Level 4
        else:  # starting_level == 4
            level3_tasks_to_create = []  # No Level 3 created
            level4_tasks_to_check = level3_finished  # Only check finished L3
    else:
        # For levels 1-2, we don't check output, so assume all tasks need Level 3
        level3_finished = []
        level3_not_finished = test_tasks
        level3_tasks_to_create = test_tasks
        level4_tasks_to_check = test_tasks
    
    # ========== LEVEL 3: Task Creation ==========
    if starting_level <= 3 and ending_level >= 3:
        for task in level3_tasks_to_create:
            global_task_name = f"GLOBAL_TC_RISK_{task['model']}_{task['variant']}_{task['scenario']}_{task['time_period']}"
            level3_tasks.append({
                'name': global_task_name,
                'model': task['model'],
                'variant': task['variant'],
                'scenario': task['scenario'],
                'time_period': task['time_period'],
            })
            level3_tasks_dict[(task['model'], task['variant'], task['scenario'], task['time_period'])] = global_task_name
            
            # Dependency: Level 3 depends on Level 2
            if ADD_DEPENDENCIES and starting_level <= 2:
                for variable in rfc.VARIABLES[DATA_SOURCE]:
                    process_task_name = f"PROCESS_{task['model']}_{task['variant']}_{task['scenario']}_{task['time_period']}_{variable}"
                    report['dependencies'].append({
                        'child': global_task_name,
                        'parent': process_task_name,
                        'level': 'L3 -> L2'
                    })
        
        report['tasks_by_level'][3]['count'] = len(level3_tasks)
        report['tasks_by_level'][3]['tasks'] = level3_tasks
    
    # ========== LEVEL 4: Check Basin Status & Clean ==========
    if starting_level <= 4 and ending_level >= 4:
        cleaned_count = 0
        incomplete_count = 0
        
        for task in level4_tasks_to_check:
            model, variant, scenario, time_period = task['model'], task['variant'], task['scenario'], task['time_period']
            
            for basin in BASINS:
                file_count = get_level4_basin_file_count(model, variant, scenario, time_period, basin, DATA_SOURCE)
                
                if file_count < NUM_DRAWS + 1:
                    incomplete_count += 1
                    
                    basin_task_name = f"BASIN_TC_RISK_{model}_{variant}_{scenario}_{time_period}_{basin}"
                    level4_basin_tasks.append({
                        'name': basin_task_name,
                        'model': model,
                        'variant': variant,
                        'scenario': scenario,
                        'time_period': time_period,
                        'basin': basin,
                        'file_count': file_count,
                    })
                    
                    # Record basin folder cleaning if enabled
                    if CLEAN_BASIN_FOLDERS_BEFORE_LEVEL4 and file_count >= 0:
                        basin_path = TC_RISK_OUTPUT_PATH / DATA_SOURCE / model / variant / scenario / time_period / basin
                        report['tasks_by_level'][4]['cleaned_basins'].append(str(basin_path))
                        cleaned_count += 1
                    
                    # Dependency: Level 4 depends on Level 3
                    if ADD_DEPENDENCIES and starting_level <= 3:
                        level3_key = (model, variant, scenario, time_period)
                        if level3_key in level3_tasks_dict:
                            report['dependencies'].append({
                                'child': basin_task_name,
                                'parent': level3_tasks_dict[level3_key],
                                'level': 'L4 -> L3'
                            })
        
        report['tasks_by_level'][4]['count'] = len(level4_basin_tasks)
        report['tasks_by_level'][4]['tasks'] = level4_basin_tasks
        report['level4_basin_status']['incomplete'] = incomplete_count
        report['level4_basin_status']['complete'] = (len(level4_tasks_to_check) * len(BASINS)) - incomplete_count
        report['tasks_by_level'][4]['cleaned_count'] = cleaned_count
    
    return report

print("Simulation function defined!")

Simulation function defined!


In [8]:
# Test all combinations of STARTING_LEVEL and ENDING_LEVEL

all_reports = {}

for starting in range(1, 5):
    for ending in range(starting, 5):
        if starting <= ending:
            key = f"L{starting}_to_L{ending}"
            all_reports[key] = simulate_orchestrator(starting, ending)
            print(f"✓ Simulated {key}")

print(f"\nCompleted {len(all_reports)} scenario combinations")

✓ Simulated L1_to_L1
✓ Simulated L1_to_L2
✓ Simulated L1_to_L3
✓ Simulated L1_to_L4
✓ Simulated L2_to_L2
✓ Simulated L2_to_L3
✓ Simulated L2_to_L4
✓ Simulated L3_to_L3
✓ Simulated L3_to_L4
✓ Simulated L4_to_L4

Completed 10 scenario combinations


In [9]:
# Display summary for each combination

import pandas as pd

summary_data = []

for scenario_key, report in all_reports.items():
    summary_data.append({
        'Scenario': scenario_key,
        'L1 Tasks': report['tasks_by_level'][1]['count'],
        'L2 Tasks': report['tasks_by_level'][2]['count'],
        'L3 Tasks': report['tasks_by_level'][3]['count'],
        'L4 Tasks': report['tasks_by_level'][4]['count'],
        'L4 Cleaned': report['tasks_by_level'][4].get('cleaned_count', 0),
        'Dependencies': len(report['dependencies']),
        'Total Folder Ops': len(report['folder_operations']),
    })

summary_df = pd.DataFrame(summary_data)
print("\n" + "="*100)
print("SUMMARY: TASK COUNTS BY SCENARIO")
print("="*100)
print(summary_df.to_string(index=False))
print()


SUMMARY: TASK COUNTS BY SCENARIO
Scenario  L1 Tasks  L2 Tasks  L3 Tasks  L4 Tasks  L4 Cleaned  Dependencies  Total Folder Ops
L1_to_L1         2         0         0         0           0             0                 4
L1_to_L2         2        12         0         0           0            12                 4
L1_to_L3         2        12         2         0           0            24                 4
L1_to_L4         2        12         2        14          14            38                 4
L2_to_L2         0        12         0         0           0             0                 0
L2_to_L3         0        12         2         0           0            12                 0
L2_to_L4         0        12         2        14          14            26                 0
L3_to_L3         0         0         0         0           0             0                 0
L3_to_L4         0         0         0        14          14             0                 0
L4_to_L4         0         0        

In [10]:
# Detailed breakdown for each scenario

for scenario_key in sorted(all_reports.keys()):
    report = all_reports[scenario_key]
    
    print("\n" + "="*100)
    print(f"SCENARIO: {scenario_key} (STARTING_LEVEL={report['starting_level']}, ENDING_LEVEL={report['ending_level']})")
    print("="*100)
    
    # Level 3 status (only relevant if starting >= 3)
    if report['starting_level'] >= 3:
        print(f"\nLevel 3 Output Status:")
        print(f"  Finished (both env_wnd + thermo):     {report['level3_status']['finished']}")
        print(f"  Not finished:                          {report['level3_status']['not_finished']}")
    
    # Tasks by level
    print(f"\nTasks Created:")
    for level in range(1, 5):
        count = report['tasks_by_level'][level]['count']
        if count > 0:
            print(f"  Level {level}: {count} task{'s' if count != 1 else ''}")
    
    # Folder operations
    if report['folder_operations']:
        print(f"\nFolder Operations ({len(report['folder_operations'])} total):")
        for op in report['folder_operations'][:5]:  # Show first 5
            print(f"  {op}")
        if len(report['folder_operations']) > 5:
            print(f"  ... and {len(report['folder_operations']) - 5} more")
    
    # Basin cleaning
    if report['tasks_by_level'][4]['cleaned_basins']:
        print(f"\nBasin Folders to Clean ({len(report['tasks_by_level'][4]['cleaned_basins'])} total):")
        for basin_path in report['tasks_by_level'][4]['cleaned_basins'][:5]:  # Show first 5
            print(f"  DELETE: {basin_path}")
        if len(report['tasks_by_level'][4]['cleaned_basins']) > 5:
            print(f"  ... and {len(report['tasks_by_level'][4]['cleaned_basins']) - 5} more")
    
    # Dependencies
    if report['dependencies']:
        print(f"\nDependencies ({len(report['dependencies'])} total):")
        # Group by level
        dep_by_level = {}
        for dep in report['dependencies']:
            level = dep['level']
            if level not in dep_by_level:
                dep_by_level[level] = 0
            dep_by_level[level] += 1
        
        for level_pair in sorted(dep_by_level.keys()):
            count = dep_by_level[level_pair]
            print(f"  {level_pair}: {count} dependency link{'s' if count != 1 else ''}")
    
    print()


SCENARIO: L1_to_L1 (STARTING_LEVEL=1, ENDING_LEVEL=1)

Tasks Created:
  Level 1: 2 tasks

Folder Operations (4 total):
  CREATE: /mnt/team/rapidresponse/pub/tropical-storms/tc_risk/input/cmip6/CMCC-ESM2/r1i1p1f1/historical/1970-1986
  CREATE: /mnt/team/rapidresponse/pub/tropical-storms/tc_risk/output/cmip6/CMCC-ESM2/r1i1p1f1/historical/1970-1986
  CREATE: /mnt/team/rapidresponse/pub/tropical-storms/tc_risk/input/cmip6/CMCC-ESM2/r1i1p1f1/historical/1987-2001
  CREATE: /mnt/team/rapidresponse/pub/tropical-storms/tc_risk/output/cmip6/CMCC-ESM2/r1i1p1f1/historical/1987-2001


SCENARIO: L1_to_L2 (STARTING_LEVEL=1, ENDING_LEVEL=2)

Tasks Created:
  Level 1: 2 tasks
  Level 2: 12 tasks

Folder Operations (4 total):
  CREATE: /mnt/team/rapidresponse/pub/tropical-storms/tc_risk/input/cmip6/CMCC-ESM2/r1i1p1f1/historical/1970-1986
  CREATE: /mnt/team/rapidresponse/pub/tropical-storms/tc_risk/output/cmip6/CMCC-ESM2/r1i1p1f1/historical/1970-1986
  CREATE: /mnt/team/rapidresponse/pub/tropical-storm

In [11]:
# Visual dependency graphs for key scenarios

import textwrap

def print_dependency_graph(scenario_key, report):
    """
    Print a visual representation of the task execution flow and dependencies.
    """
    print(f"\n{'='*100}")
    print(f"DEPENDENCY GRAPH: {scenario_key}")
    print(f"{'='*100}\n")
    
    start_level = report['starting_level']
    end_level = report['ending_level']
    
    # Print execution flow
    print("Execution Flow:")
    flow_levels = []
    for level in range(1, 5):
        count = report['tasks_by_level'][level]['count']
        if start_level <= level <= end_level:
            if count > 0:
                flow_levels.append(f"Level {level} ({count} tasks)")
            else:
                flow_levels.append(f"Level {level} (SKIPPED - no tasks)")
        else:
            flow_levels.append(f"Level {level} (NOT RUN)")
    
    for i, level_str in enumerate(flow_levels):
        print(f"  {level_str}")
        if i < len(flow_levels) - 1:
            print(f"    ↓")
    
    # Print key operations
    print(f"\nKey Operations:")
    
    if report['folder_operations']:
        print(f"\n  1. FOLDER CREATION (Level 1)")
        print(f"     Creates input and output directories for each task")
        for i, op in enumerate(report['folder_operations'][:2]):
            path = op.replace('CREATE: ', '')
            short_path = '/'.join(path.split('/')[-3:])
            print(f"       • {short_path}")
        if len(report['folder_operations']) > 2:
            print(f"       • ... ({len(report['folder_operations']) - 2} more)")
    
    if report['tasks_by_level'][2]['count'] > 0:
        print(f"\n  2. PROCESS VARIABLES (Level 2)")
        print(f"     Processes raw data for {len(rfc.VARIABLES[DATA_SOURCE])} variables per task")
        print(f"     Total: {report['tasks_by_level'][2]['count']} tasks")
    
    if report['tasks_by_level'][3]['count'] > 0:
        print(f"\n  3. RUN GLOBAL TC-RISK (Level 3)")
        print(f"     Computes global TC-risk metrics")
        print(f"     Total: {report['tasks_by_level'][3]['count']} tasks")
    
    if report['tasks_by_level'][4]['cleaned_basins']:
        print(f"\n  4a. CLEAN BASIN FOLDERS (before Level 4)")
        print(f"     Deletes all .nc files in incomplete basin folders")
        print(f"     Total folders cleaned: {len(report['tasks_by_level'][4]['cleaned_basins'])}")
    
    if report['tasks_by_level'][4]['count'] > 0:
        print(f"\n  4b. RUN BASIN TC-RISK (Level 4)")
        print(f"     Computes basin-specific TC-risk metrics")
        print(f"     Total: {report['tasks_by_level'][4]['count']} tasks")
        print(f"     ⚠️  Requires >= {NUM_DRAWS + 1} files per basin")
    
    # Print dependency summary
    if report['dependencies']:
        print(f"\nDependency Summary:")
        dep_summary = {}
        for dep in report['dependencies']:
            level_pair = dep['level']
            if level_pair not in dep_summary:
                dep_summary[level_pair] = 0
            dep_summary[level_pair] += 1
        
        for level_pair in sorted(dep_summary.keys()):
            print(f"  {level_pair}: {dep_summary[level_pair]} links")


# Print graphs for key scenarios
key_scenarios = ['L1_to_L4', 'L2_to_L4', 'L3_to_L4', 'L3_to_L3', 'L4_to_L4']
for scenario in key_scenarios:
    if scenario in all_reports:
        print_dependency_graph(scenario, all_reports[scenario])


DEPENDENCY GRAPH: L1_to_L4

Execution Flow:
  Level 1 (2 tasks)
    ↓
  Level 2 (12 tasks)
    ↓
  Level 3 (2 tasks)
    ↓
  Level 4 (14 tasks)

Key Operations:

  1. FOLDER CREATION (Level 1)
     Creates input and output directories for each task
       • r1i1p1f1/historical/1970-1986
       • r1i1p1f1/historical/1970-1986
       • ... (2 more)

  2. PROCESS VARIABLES (Level 2)
     Processes raw data for 6 variables per task
     Total: 12 tasks

  3. RUN GLOBAL TC-RISK (Level 3)
     Computes global TC-risk metrics
     Total: 2 tasks

  4a. CLEAN BASIN FOLDERS (before Level 4)
     Deletes all .nc files in incomplete basin folders
     Total folders cleaned: 14

  4b. RUN BASIN TC-RISK (Level 4)
     Computes basin-specific TC-risk metrics
     Total: 14 tasks
     ⚠️  Requires >= 251 files per basin

Dependency Summary:
  L2 -> L1: 12 links
  L3 -> L2: 12 links
  L4 -> L3: 14 links

DEPENDENCY GRAPH: L2_to_L4

Execution Flow:
  Level 1 (NOT RUN)
    ↓
  Level 2 (12 tasks)
    ↓


In [12]:
# Detailed example: Show actual task names and dependencies for L3_to_L4

print("\n" + "="*100)
print("DETAILED EXAMPLE: L3_to_L4 (Most Common Use Case)")
print("="*100)

report = all_reports['L3_to_L4']

print(f"\n📊 CONFIGURATION")
print(f"  STARTING_LEVEL: {report['starting_level']}")
print(f"  ENDING_LEVEL: {report['ending_level']}")
print(f"  ADD_DEPENDENCIES: True")
print(f"  CLEAN_BASIN_FOLDERS_BEFORE_LEVEL4: {CLEAN_BASIN_FOLDERS_BEFORE_LEVEL4}")

print(f"\n📋 LEVEL 3 OUTPUT STATUS (CHECK ONLY, NO CREATION)")
print(f"  Tasks with complete L3 output (both env_wnd + thermo): {report['level3_status']['finished']}")
print(f"  Tasks with incomplete L3 output: {report['level3_status']['not_finished']}")

print(f"\n✅ TASKS TO CREATE")
for level in range(1, 5):
    tasks = report['tasks_by_level'][level]['tasks']
    if tasks:
        print(f"\n  Level {level}: {len(tasks)} task{'s' if len(tasks) != 1 else ''}")
        for task in tasks[:2]:  # Show first 2
            print(f"    • {task['name']}")
        if len(tasks) > 2:
            print(f"    • ... ({len(tasks) - 2} more)")

print(f"\n🔗 DEPENDENCIES")
dep_by_type = {}
for dep in report['dependencies']:
    dep_type = dep['level']
    if dep_type not in dep_by_type:
        dep_by_type[dep_type] = []
    dep_by_type[dep_type].append(dep)

for dep_type in sorted(dep_by_type.keys()):
    deps = dep_by_type[dep_type]
    print(f"\n  {dep_type}: {len(deps)} links")
    for dep in deps[:2]:
        child_short = dep['child'].split('_')[-1] if '_' in dep['child'] else dep['child'][:20]
        parent_short = dep['parent'].split('_')[-1] if '_' in dep['parent'] else dep['parent'][:20]
        print(f"    • {child_short} ← {parent_short}")
    if len(deps) > 2:
        print(f"    • ... ({len(deps) - 2} more)")

print(f"\n🧹 CLEANING OPERATIONS (Before Level 4 execution)")
if report['tasks_by_level'][4]['cleaned_basins']:
    print(f"  Total basin folders to clean: {len(report['tasks_by_level'][4]['cleaned_basins'])}")
    for basin_path in report['tasks_by_level'][4]['cleaned_basins'][:3]:
        short_path = '/'.join(basin_path.split('/')[-5:])
        print(f"    DELETE: {short_path}")
    if len(report['tasks_by_level'][4]['cleaned_basins']) > 3:
        print(f"    ... ({len(report['tasks_by_level'][4]['cleaned_basins']) - 3} more)")
else:
    print(f"  No basin folders to clean")


DETAILED EXAMPLE: L3_to_L4 (Most Common Use Case)

📊 CONFIGURATION
  STARTING_LEVEL: 3
  ENDING_LEVEL: 4
  ADD_DEPENDENCIES: True
  CLEAN_BASIN_FOLDERS_BEFORE_LEVEL4: True

📋 LEVEL 3 OUTPUT STATUS (CHECK ONLY, NO CREATION)
  Tasks with complete L3 output (both env_wnd + thermo): 2
  Tasks with incomplete L3 output: 0

✅ TASKS TO CREATE

  Level 4: 14 tasks
    • BASIN_TC_RISK_CMCC-ESM2_r1i1p1f1_historical_1970-1986_EP
    • BASIN_TC_RISK_CMCC-ESM2_r1i1p1f1_historical_1970-1986_NA
    • ... (12 more)

🔗 DEPENDENCIES

🧹 CLEANING OPERATIONS (Before Level 4 execution)
  Total basin folders to clean: 14
    DELETE: CMCC-ESM2/r1i1p1f1/historical/1970-1986/EP
    DELETE: CMCC-ESM2/r1i1p1f1/historical/1970-1986/NA
    DELETE: CMCC-ESM2/r1i1p1f1/historical/1970-1986/NI
    ... (11 more)


In [13]:
# Special handling for STARTING_LEVEL = 3 vs 4 logic

print("\n" + "="*100)
print("SPECIAL CASE: STARTING_LEVEL = 3 (Resuming from unfinished Level 3)")
print("="*100)

report_l3 = all_reports['L3_to_L3']

print(f"\n📊 WHAT HAPPENS")
print(f"\n  1. Check all tasks for Level 3 completion (need both env_wnd + thermo files)")
print(f"     • Found {report_l3['level3_status']['finished']} tasks with finished L3")
print(f"     • Found {report_l3['level3_status']['not_finished']} tasks with incomplete L3")
print(f"\n  2. Create Level 3 tasks ONLY for the {report_l3['level3_status']['not_finished']} incomplete tasks")
print(f"     • Level 3 tasks created: {report_l3['tasks_by_level'][3]['count']}")
print(f"\n  3. When Level 3 completes, Level 4 will run WITH dependencies")
print(f"     • L4 runs for (a) basins under newly completed L3 tasks")
print(f"     • L4 runs for (b) incomplete basins under already-finished L3 tasks")

print(f"\n" + "="*100)
print(f"SPECIAL CASE: STARTING_LEVEL = 4 (Running only Level 4)")
print(f"="*100)

report_l4 = all_reports['L4_to_L4']

print(f"\n📊 WHAT HAPPENS")
print(f"\n  1. Check all tasks for Level 3 completion")
print(f"     • Found {report_l4['level3_status']['finished']} tasks with finished L3")
print(f"     • Found {report_l4['level3_status']['not_finished']} tasks with incomplete L3")
print(f"\n  2. SKIP Level 3 entirely (no tasks created)")
print(f"\n  3. Create Level 4 tasks ONLY for basins where:")
print(f"     • (a) The time-period passed the L3 test ({report_l4['level3_status']['finished']} tasks)")
print(f"     • (b) The basin has < {NUM_DRAWS + 1} files (incomplete)")
print(f"     • Level 4 tasks created: {report_l4['tasks_by_level'][4]['count']}")
print(f"\n  4. NO dependencies on Level 3 (they run independently)")


SPECIAL CASE: STARTING_LEVEL = 3 (Resuming from unfinished Level 3)

📊 WHAT HAPPENS

  1. Check all tasks for Level 3 completion (need both env_wnd + thermo files)
     • Found 2 tasks with finished L3
     • Found 0 tasks with incomplete L3

  2. Create Level 3 tasks ONLY for the 0 incomplete tasks
     • Level 3 tasks created: 0

  3. When Level 3 completes, Level 4 will run WITH dependencies
     • L4 runs for (a) basins under newly completed L3 tasks
     • L4 runs for (b) incomplete basins under already-finished L3 tasks

SPECIAL CASE: STARTING_LEVEL = 4 (Running only Level 4)

📊 WHAT HAPPENS

  1. Check all tasks for Level 3 completion
     • Found 2 tasks with finished L3
     • Found 0 tasks with incomplete L3

  2. SKIP Level 3 entirely (no tasks created)

  3. Create Level 4 tasks ONLY for basins where:
     • (a) The time-period passed the L3 test (2 tasks)
     • (b) The basin has < 251 files (incomplete)
     • Level 4 tasks created: 14

  4. NO dependencies on Level 3 (t

In [14]:
# Print all 10 possible combinations in a concise format

print("\n" + "="*100)
print("ALL 10 COMBINATIONS: CONCISE REFERENCE")
print("="*100 + "\n")

combinations = []
for starting in range(1, 5):
    for ending in range(starting, 5):
        key = f"L{starting}_to_L{ending}"
        report = all_reports[key]
        combinations.append({
            'Combination': key,
            'L1': report['tasks_by_level'][1]['count'],
            'L2': report['tasks_by_level'][2]['count'],
            'L3': report['tasks_by_level'][3]['count'],
            'L4': report['tasks_by_level'][4]['count'],
            'Dependencies': len(report['dependencies']),
            'Folder Ops': len(report['folder_operations']),
            'Basins Cleaned': report['tasks_by_level'][4].get('cleaned_count', 0),
        })

combo_df = pd.DataFrame(combinations)
print(combo_df.to_string(index=False))

print("\n" + "="*100)
print("KEY INSIGHTS")
print("="*100)
print(f"""
1. L1_to_L4: Full pipeline from scratch
   - Creates folders, processes all variables, runs L3 and L4
   - Maximum dependencies: {all_reports['L1_to_L4']['tasks_by_level'][2]['count']} (L2→L1) + {len([d for d in all_reports['L1_to_L4']['dependencies'] if 'L3' in d['level']])} (L3→L2) + {len([d for d in all_reports['L1_to_L4']['dependencies'] if 'L4' in d['level']])} (L4→L3)

2. L3_to_L4: Resume from unfinished Level 3
   - Only runs unfinished L3 tasks ({all_reports['L3_to_L4']['level3_status']['not_finished']} in test)
   - Automatically creates L4 for completed + incomplete basins
   - Basin folders are cleaned before L4 runs
   - Dependencies link L4 → L3

3. L4_to_L4: Re-run only Level 4
   - Completely skips L3 creation
   - Only checks basins under FINISHED L3 tasks
   - Only creates L4 for incomplete basins (< {NUM_DRAWS + 1} files)
   - NO dependencies (runs independently)
   - Best for re-running failed basin draws

4. Basin Cleaning:
   - Happens BEFORE Level 4 execution
   - Only if CLEAN_BASIN_FOLDERS_BEFORE_LEVEL4 = True
   - Deletes all .nc files in incomplete basin folders
   - Ensures clean slate for L4 execution
""")


ALL 10 COMBINATIONS: CONCISE REFERENCE

Combination  L1  L2  L3  L4  Dependencies  Folder Ops  Basins Cleaned
   L1_to_L1   2   0   0   0             0           4               0
   L1_to_L2   2  12   0   0            12           4               0
   L1_to_L3   2  12   2   0            24           4               0
   L1_to_L4   2  12   2  14            38           4              14
   L2_to_L2   0  12   0   0             0           0               0
   L2_to_L3   0  12   2   0            12           0               0
   L2_to_L4   0  12   2  14            26           0              14
   L3_to_L3   0   0   0   0             0           0               0
   L3_to_L4   0   0   0  14             0           0              14
   L4_to_L4   0   0   0  14             0           0              14

KEY INSIGHTS

1. L1_to_L4: Full pipeline from scratch
   - Creates folders, processes all variables, runs L3 and L4
   - Maximum dependencies: 12 (L2→L1) + 26 (L3→L2) + 14 (L4→L3)

2. L3_t

In [15]:
# Summary of exact execution order and operations

print("\n" + "="*100)
print("EXECUTION ORDER AND OPERATIONS BY SCENARIO")
print("="*100)

execution_orders = {
    'L1_to_L1': [
        "1. Create input/output folders",
        "STOP (no further levels)"
    ],
    'L1_to_L2': [
        "1. Create input/output folders",
        "2. Process variables (all 6 variables per task)",
        "STOP (no further levels)"
    ],
    'L1_to_L3': [
        "1. Create input/output folders",
        "2. Process variables (all 6 variables per task)",
        "3. Run global TC-risk",
        "STOP (no further levels)"
    ],
    'L1_to_L4': [
        "1. Create input/output folders",
        "2. Process variables (all 6 variables per task)",
        "3. Run global TC-risk",
        "4. Clean incomplete basin folders (if enabled)",
        "5. Run basin TC-risk (one per basin, with L3 dependency)",
        "Done"
    ],
    'L2_to_L2': [
        "1. SKIP folder creation",
        "2. Process variables (all 6 variables per task)",
        "STOP (no further levels)"
    ],
    'L2_to_L3': [
        "1. SKIP folder creation",
        "2. Process variables (all 6 variables per task)",
        "3. Run global TC-risk",
        "STOP (no further levels)"
    ],
    'L2_to_L4': [
        "1. SKIP folder creation",
        "2. Process variables (all 6 variables per task)",
        "3. Run global TC-risk",
        "4. Clean incomplete basin folders (if enabled)",
        "5. Run basin TC-risk (one per basin, with L3 dependency)",
        "Done"
    ],
    'L3_to_L3': [
        "1. Check L3 output status for all tasks",
        "2. SKIP folder creation and L2 processing",
        "3. Run global TC-risk ONLY for unfinished tasks",
        "STOP (no further levels)"
    ],
    'L3_to_L4': [
        "1. Check L3 output status for all tasks",
        "2. SKIP folder creation and L2 processing",
        "3. Run global TC-risk ONLY for unfinished tasks",
        "4. Clean incomplete basin folders (if enabled)",
        "5. Run basin TC-risk for incomplete basins (with L3 dependency)",
        "Done"
    ],
    'L4_to_L4': [
        "1. Check L3 output status for all tasks",
        "2. SKIP L1, L2, L3 entirely",
        "3. Check which basins have < " + str(NUM_DRAWS + 1) + " files",
        "4. Clean incomplete basin folders (if enabled)",
        "5. Run basin TC-risk ONLY for incomplete basins under finished L3",
        "6. NO dependencies on L3 (independent execution)",
        "Done"
    ]
}

for scenario in sorted(execution_orders.keys()):
    print(f"\n{scenario.upper()}:")
    for step in execution_orders[scenario]:
        if step.startswith('STOP') or step.startswith('Done'):
            print(f"  ⛔ {step}")
        elif 'SKIP' in step:
            print(f"  ⊘ {step}")
        elif 'Check' in step:
            print(f"  🔍 {step}")
        elif 'Clean' in step:
            print(f"  🧹 {step}")
        else:
            print(f"  ✓ {step}")


EXECUTION ORDER AND OPERATIONS BY SCENARIO

L1_TO_L1:
  ✓ 1. Create input/output folders
  ⛔ STOP (no further levels)

L1_TO_L2:
  ✓ 1. Create input/output folders
  ✓ 2. Process variables (all 6 variables per task)
  ⛔ STOP (no further levels)

L1_TO_L3:
  ✓ 1. Create input/output folders
  ✓ 2. Process variables (all 6 variables per task)
  ✓ 3. Run global TC-risk
  ⛔ STOP (no further levels)

L1_TO_L4:
  ✓ 1. Create input/output folders
  ✓ 2. Process variables (all 6 variables per task)
  ✓ 3. Run global TC-risk
  🧹 4. Clean incomplete basin folders (if enabled)
  ✓ 5. Run basin TC-risk (one per basin, with L3 dependency)
  ⛔ Done

L2_TO_L2:
  ⊘ 1. SKIP folder creation
  ✓ 2. Process variables (all 6 variables per task)
  ⛔ STOP (no further levels)

L2_TO_L3:
  ⊘ 1. SKIP folder creation
  ✓ 2. Process variables (all 6 variables per task)
  ✓ 3. Run global TC-risk
  ⛔ STOP (no further levels)

L2_TO_L4:
  ⊘ 1. SKIP folder creation
  ✓ 2. Process variables (all 6 variables per task)